# Black-Owned Businesses in Boston

In [163]:
from geopy.geocoders import Nominatim
import pandas as pd
import folium
import time

In [232]:
restaurant = pd.read_csv('/Users/avonleafisher/Desktop/Restaurants.csv')
store = pd.read_csv('/Users/avonleafisher/Desktop/Stores.csv')
service = pd.read_csv('/Users/avonleafisher/Desktop/Services.csv')

## Restaurants

In [165]:
restaurant = restaurant.loc[3:]
header = restaurant.iloc[0] 
restaurant = restaurant[1:] 
restaurant.columns = header
restaurant.drop('#', axis=1, inplace=True)
restaurant = restaurant[restaurant['Street Address'].isna() == False]

In [166]:
restaurant = restaurant[['Name', 'Black Owned Confirmed?', 'Street Address', 'City', 'State', 'Zipcode']]
restaurant.reset_index(inplace=True)
restaurant.isna().sum()

3
index                     0
Name                      0
Black Owned Confirmed?    0
Street Address            0
City                      1
State                     0
Zipcode                   5
dtype: int64

In [167]:
restaurant['Street Address'] = [i.title() for i in restaurant['Street Address']]

In [168]:
restaurant['Street Address'][94] = "5197 Washington Street"
restaurant['Street Address'][53] = "233 River Street"
restaurant['Street Address'][89] = "285 Beacon Street"

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thi

In [169]:
restaurant['Street Address'] = [i + ',' for i in restaurant['Street Address']]
restaurant['City'] = [str(i) + ',' for i in restaurant['City']]
restaurant['Address'] = restaurant['Street Address'].str.cat(restaurant[['City', 'State', 'Zipcode']], sep=' ')
restaurant = restaurant.drop(['index', 'Street Address', 'City', 'State', 'Zipcode'], axis=1)

In [170]:
restaurant.Address

0            568 Cambridge St, Allston, MA 02134
1                  490 W Main St, Avon, MA 02322
2               266 Bowdoin St, Boston, MA 02122
3          2255 Dorchester Ave, Boston, MA 02124
4             604 Columbus Ave, Boston, MA 02118
                         ...                    
94    5197 Washington Street, MA 02132, MA 02132
95           157 Belmont St, Worcester, MA 01605
96                                           NaN
97          687 Millbury St, Worcester, MA 01607
98           657 Washington St, Boston, MA 02124
Name: Address, Length: 99, dtype: object

In [171]:
geolocator = Nominatim(user_agent='fisheravonlea@gmail.com')

In [177]:
def get_lat_lon(address_column):
    lat = []
    lon = []
    for a in address_column:
        location = geolocator.geocode(a)
        if location is not None:
            lat.append(location.latitude)
            lon.append(location.longitude)
        else:
            lat.append('NA')
            lon.append('NA')
    time.sleep(1)
    
    return lat, lon

In [179]:
lat, lon = get_lat_lon(restaurant.Address)
restaurant['Lat'] = lat
restaurant['Lon'] = lon

In [181]:
restaurant.head()

3,Name,Black Owned Confirmed?,Address,Lat,Lon
0,Rock City Pizza,Confirmed,"568 Cambridge St, Allston, MA 02134",42.3523,-71.1406
1,TamBos Kitchen,Unconfirmed,"490 W Main St, Avon, MA 02322",42.1233,-71.0409
2,Restaurante Cesaria,Confirmed,"266 Bowdoin St, Boston, MA 02122",42.3069,-71.0664
3,Bred Gourmet,Confirmed,"2255 Dorchester Ave, Boston, MA 02124",42.2735,-71.0678
4,Darryl's Corner Bar and Kitchen,Confirmed,"604 Columbus Ave, Boston, MA 02118",42.3398,-71.0825


In [182]:
restaurant = restaurant[restaurant.Lat != 'NA']

In [202]:
basemap = folium.Map(location=[42.361145, -71.057083], tiles="Stamen Toner", zoom_start=12)

In [204]:
def add_markers(df, color, icon):
    
    x = df.Lat
    y = df.Lon
    Name = df.Name
    Address = df.Address
    
    points = list(zip(x, y, Name, Address))
    for p in points:
        popup_text = "{}, {}".format(str(p[2]), str(p[3]))
        popup = folium.Popup(popup_text, autopan='False',parse_html=True)
        
        marker = folium.Marker(location=[p[0], p[1]], 
                                popup=popup, 
                                icon = folium.Icon(icon_size=(27, 27),
                                                      color=color,
                                                      icon=icon),
                                    ).add_to(basemap)

In [205]:
add_markers(restaurant, 'green', 'cutlery')

In [206]:
basemap

## Stores

In [212]:
store.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,#,NAME,Industry,LOCATION,WEBSITE,TELEPHONE,EMAIL,Combined ADDRESS,ADDRESS,Zipcode
1,1,RepHAIRations,NaN,Arlington,https://www.rephairations.com/,(781) 777-1527,info@rephairations.com,"1339 Massachusetts Avenue Arlington, MA 02476","1339 Massachusetts Avenue Arlington, MA 02476",02476
2,2,Mamadou' Artisan Bakery,NaN,Arlington & Winchester,https://www.yelp.com/biz/mamadous-artisan-bake...,NaN,NaN,"677 Massachusetts Ave Arlington, MA 02476 Arli...","677 Massachusetts Ave Arlington, MA 02476 Arli...",NaN
3,3,Elite Lash Studio,NaN,Back Bay,http://elitelashstudio.com,617 262-5274,info@elitelashstudio.com,"224 Clarendon Street, Suite 51 Boston, MA 02116","224 Clarendon Street, Suite 51 Boston, MA 02116",02116
4,4,Antisocial Skate Club,NaN,Boston,https://officialblackwallstreet.com/biz/antiso...,NaN,NaN,NaN,NaN,NaN


In [213]:
header = store.iloc[0] 
store = store[1:] 
store.columns = header
store = store[store['ADDRESS'].isna() == False]

In [214]:
store = store[['NAME', 'ADDRESS']]
store.columns = ['Name', 'Address']

In [215]:
store

,Name,Address
1,RepHAIRations,"1339 Massachusetts Avenue Arlington, MA 02476"
2,Mamadou' Artisan Bakery,"677 Massachusetts Ave Arlington, MA 02476 Arli..."
3,Elite Lash Studio,"224 Clarendon Street, Suite 51 Boston, MA 02116"
5,Bebe Mursalin,"32 Province Street, Boston, MA 02108"
6,Boston Beauty Bar,"388 Washington St, Boston, MA 02108"
7,Brown and Coconut,Available at both Cambridge Naturals locations...
8,Dartmouth Street Vision Center,130 Dartmouth St (across the street from Bosto...
18,Hair Sessions,"437 Boylston St, Boston, MA 02116"
19,M Salon,"54 Queensbury St, Boston, MA 02215"
20,Just the Best Real Estate/Mass Affordable Hous...,Facebook.com/Alexjbrealestate


In [217]:
lat, lon = get_lat_lon(store.Address)
store['Lat'] = lat
store['Lon'] = lon
store = store[store.Lat != 'NA']

In [219]:
store

,Name,Address,Lat,Lon
1,RepHAIRations,"1339 Massachusetts Avenue Arlington, MA 02476",42.4245,-71.1835
2,Mamadou' Artisan Bakery,"677 Massachusetts Ave Arlington, MA 02476 Arli...",42.4223,-71.1732
5,Bebe Mursalin,"32 Province Street, Boston, MA 02108",42.357,-71.0598
6,Boston Beauty Bar,"388 Washington St, Boston, MA 02108",42.3564,-71.0592
18,Hair Sessions,"437 Boylston St, Boston, MA 02116",42.3515,-71.0729
21,L'Accent Women's Fashion,"395 A Washington Street, Brookline, MA 02446",42.3348,-71.1232
24,Million Year Picnic (Comic Books),"99 Mount Auburn St, Cambridge, MA 02138",42.3729,-71.1206
25,Nu Image Barbershop,"98 River Street Cambridge, MA 02139",42.3654,-71.104
26,La Chic Dog Grooming Salon,"316 Main St, Charlestown, MA 02129",42.3783,-71.068
32,Exodus Bagels,"3346 Washington St, Boston, MA 02130",42.3095,-71.1044


In [223]:
add_markers(store, 'purple', 'usd')

In [224]:
basemap

## Services

In [228]:
services.head()

,Source: https://blackboston.com/businesslist/ list purchased on 6.2.2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,(Minority Business Enterprise) MBE,(Women Business Enterprise) WBE,Category,Company Name,Address,City,State,Zip code,County,First name,Last Name,Phone,Fax,Email,Website,(Public Business Entity) PBE - Y/N,Non Profit - Y/N,Cert Date,Profile of Service Description
1,Y,Y,EN - ENVIRONMENTAL CONSULTANTS,Atkinson-Whitworth Corporation d/b/a: Jet-A-Way,"1477 Bedford St., 2nd Floor",Abington,MA,02351,Suffolk,Darlene,Jeter,(617) 541-4009,(617) 541-4011,djeter@jawdirect.com,http://www.jawdirect.com,N,N,02/01/1982,"Rubbish Removal, Waste Hauling, General Trucki..."
2,Y,Y,TK - TRUCKING,Atkinson-Whitworth Corporation d/b/a: Jet-A-Way,"1477 Bedford St., 2nd Floor",Abington,MA,02351,Suffolk,Darlene,Jeter,(617) 541-4009,(617) 541-4011,djeter@jawdirect.com,http://www.jawdirect.com,N,N,02/01/1982,"Rubbish Removal, Waste Hauling, General Trucki..."
3,Y,N,CO - CONSULTANTS,"Peakpoint Technologies, Inc.","930 Main Street, Suite 103",Acton,MA,01720,Middlesex,Senthil,Kumar,(330) 732-5768,(815) 927-0204,senthil@bigfoot.com,NaN,N,N,09/04/2002,Development and Maintenance of Database and So...
4,Y,N,CP - COMPUTER SOFTWARE,"Peakpoint Technologies, Inc.","930 Main Street, Suite 103",Acton,MA,01720,Middlesex,Senthil,Kumar,(330) 732-5768,(815) 927-0204,senthil@bigfoot.com,NaN,N,N,09/04/2002,Development and Maintenance of Database and So...


In [233]:
header = service.iloc[0] 
service = service[1:] 
service.columns = header
service = service[service['Address'].isna() == False]

In [236]:
service = service[['Company Name', 'Address']]

In [237]:
service.columns = ['Name', 'Address']

In [241]:
service = service.drop_duplicates()

In [244]:
service[20:40]

,Name,Address
57,Bredy Network Management Corporation,50 Union Street
58,"Intergeneration Capital Management, LLC","300 Brickstone Square, Suite 201"
59,"PMA Consultants, LLC",226 West Liberty
61,"Encore Consulting Services, Inc.","116 S. Arlington Heights Road, Suite 301"
66,E-Squared Consulting Corporation d/b/a: E-Squa...,"43969 Tavern Drive, #200"
70,"RRK Consulting, LLC",4 Wadsworth Road
73,"HiFi Technologies, Inc.",8 Annetta Road
75,"VP Engineering, Inc.","4080 Conway Valley Road, NW"
76,"CodeRed Business Solutions, Inc.",966 Westcott Lane
77,Todd McGhee d/b/a: 508 Group,48 Farmers Lane
